In [27]:
import sys
import requests
!{sys.executable} -m pip install fake_useragent
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
import csv
from itertools import chain

# Step 1. Collecting data

Я буду использовать пользовательсие отзывы на анимацию на метакритике.

In [7]:
session = requests.session()

In [112]:
# collecting individual film pages
def page(url):
    u_a = UserAgent(verify_ssl=False)
    response = session.get(url, headers={'User-Agent': u_a.random})
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', {'class': 'metascore_anchor'})
    links = list(map(lambda x: 'https://www.metacritic.com' + x.attrs['href'], links))
    return links[::3]

In [86]:
links = ['https://www.metacritic.com/browse/movies/genre/userscore/animation?view=detailed', 'https://www.metacritic.com/browse/movies/genre/userscore/animation?view=detailed&page=5']

In [87]:
pages = list(chain(*list(map(page, links))))

In [125]:
# collecting reviews and corresponding scores
def film(url):
    u_a = UserAgent(verify_ssl=False)
    response = session.get(url, headers={'User-Agent': u_a.random})
    soup = BeautifulSoup(response.text, 'html.parser')
    reviews = soup.find_all('span', {'class': 'blurb blurb_expanded'})
    cl = re.compile("metascore_w user(.+)")
    marks = soup.find_all('div', {'class':cl})
    try:
        reviews = list(map(lambda x: re.sub(r'<br/>', ' ', str(x)), reviews))
        reviews = list(map(lambda x: re.search(r'ed">(.+?)</span>', x).group(1), reviews))
        marks = list(map(lambda x: float(x.text), marks))
    except:
        return ('', 0)
    return list(zip(marks, reviews))

In [128]:
# flattening out
pairs = list(chain(*list(map(film, pages))))
clean_pairs = list(filter(lambda x: type(x)==tuple, pairs))

In [129]:
# writing down
with open('reviews.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=' ')
    for rr in clean_pairs:
        writer.writerow(list(rr))

In [16]:
#shortcut reading out of  file in case of restart
with open('reviews.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=' ')
    clean_pairs = list(filter(lambda x: len(x) == 2, [*reader]))

In [17]:
fin_pairs = sorted(clean_pairs, key=lambda x: x[0])

In [113]:
fin_pairs[1]

['0.0',
 "The universal acclaim about this movie is to be understood by the atmosphere it creates. Everywhere your look goes you see bright colors and movements and animations you simply can't understand. Because 'Promare' definitively is aggressive, moving and clocked. Characters are attractive, intelligently connected an funny. You can find differents sub-stories that evolve in parallel with the main plot and those add some content to the characters it concerns without dragging us out of the movie. Far from being perfect, visual effects do bring a lot in 'Promare' and there is something charming when you watch it. However, story is way too conventional and there is no turn of events unless those who are fundamental to japan-animation movies. I enjoyed to discover the universe in which Promare people live but I was less intrigued by the resolution that you can predict from miles and miles away. Animation is colorful yes but jerky and often really hard to follow. Shapes and colors don'

In [19]:
# separating
good_revs = list(map(lambda x: x[1], fin_pairs[-210:]))
bad_revs = list(map(lambda x: x[1], fin_pairs[:210]))

# Step 2. Lemmatization

In [20]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [21]:
def clean_up(pp):
    tok = word_tokenize(pp.lower())
    depunct = list(filter(lambda x: x not in punctuation, tok))
    lemmer = WordNetLemmatizer()
    lems = list(map(lambda x: lemmer.lemmatize(x), depunct))
    return lems

In [22]:
g_lems = list(map(clean_up, good_revs))

In [23]:
b_lems = list(map(clean_up, bad_revs))

In [24]:
# separating testing samples
g_lems_ch = g_lems[:10]
g_lems = g_lems[10:]
b_lems_ch = b_lems[:10]
b_lems = b_lems[10:]

# Step 3. Word-lists

In [25]:
from collections import Counter

In [28]:
g_lems = list(chain(*g_lems))

In [29]:
b_lems = list(chain(*b_lems))

In [30]:
gco = Counter(g_lems)
bco = Counter(b_lems)

In [31]:
def not_in(l1,l2):  #elements of l1 that are not in l2
    return list(filter(lambda x: x not in l2.keys(), list(l1.keys())))

In [32]:
uniq_g = not_in(gco, bco)
uniq_b = not_in(bco, gco)

In [33]:
#only in good + more frequent than 2 times
freq_uniq_g = list(filter(lambda x: gco[x]>2, uniq_g))
#only in bad + more frequent than 2 times
freq_uniq_b = list(filter(lambda x: bco[x]>2, uniq_b))

In [36]:
# how many words were cut off by frequency?
len(freq_uniq_g)

275

In [35]:
len(uniq_g)

2778

# Step 4. +/- function

In [91]:
def detector(l_words, g_list,  b_list):  # gets a preprocessed list of words of a comment+ freqlists
    goods = list(filter(lambda x: x in g_list, l_words))
    bads = list(filter(lambda x: x in b_list, l_words))
    if len(goods) < len(bads):
            return -1
    else:
        return 1

In [86]:
detector(g_lems_ch[5], uniq_g, uniq_b)

1

In [87]:
!{sys.executable} -m pip install sklearn

In [88]:
from sklearn.metrics import accuracy_score

In [89]:
answ = [1]*10 + [-1]*10
datas = g_lems_ch + b_lems_ch

In [92]:
# no frequency threshold
predict = list(map(lambda x: detector(x, uniq_g, uniq_b), datas))
accuracy_score(predict, answ)

0.6

In [93]:
#  frequencies >2
predict = list(map(lambda x: detector(x, freq_uniq_g, freq_uniq_b), datas))
accuracy_score(predict, answ)

0.55

# Step 5. Приблуды

1) Хочется попробовать использовать только оценочные слова - с наибольшей вероятностью это прилагательные.
Попробую выделить только их через pos-тэги

In [95]:
from nltk import pos_tag

In [98]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Swl\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [105]:
def poss(wrds):
    return [x[0] for x in pos_tag(wrds) if x[1]=='JJ']

In [106]:
poss(datas[0])

['great', 'kinda', 'great', 'i', 'i', 'black', 'memorable', 'great', 'cool']

In [110]:
# no frequency threshold + adj-filter
pos_datas = list(map(poss, datas))
predict = list(map(lambda x: detector(x, uniq_g, uniq_b), pos_datas))
accuracy_score(predict, answ)

0.45

In [111]:
#  frequencies >2 + adj-filter
predict = list(map(lambda x: detector(x, freq_uniq_g, freq_uniq_b), pos_datas))
accuracy_score(predict, answ)

0.45

Стало хуже.
2) Можно попробовать почистить датасет. Во-первых, там есть несколько отзывов на итальянском - можно убрать их пакетным определителем языка каким-то. Во-вторых, судя по всему, некоторые пользоватеи забывали ставить оценку в системе и ставили её только текстом в отзыве - из-за этого есть некоторое количество отзывов с оценкой 0.0, но с положительным текстом, в котором эксплицитно написано 8/10. Можно попробовать искать цифры в отзывах и отсеивать такие выбросы.